<a href="https://colab.research.google.com/github/yinyang-boop/MM-DLP-DR/blob/main/MM_DLP_DR_based_on_chEMBL%2C_DrugBank%2C_GSDC_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: Initial Colab Configuration

In [ ]:
# Enable GPU acceleration
# Go to Runtime; Change runtime type; Hardware accelerator; GPU/T4
# Mount Google Drive for data persistence
from google.colab import drive
drive.mount('/content/gdrive')
# Create project directory
import os
project_dir = '/content/gdrive/MyDrive/DDLS_Drug_Repurposing'
os.makedirs(project_dir, exist_ok=True)
os.chdir(project_dir)
# Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
print(f"CUDA version: {torch.version.cuda}")
print(f"GPU device: {torch.cuda.get_device_name(0)}")

IndentationError: expected an indented block after 'if' statement on line 15 (ipython-input-2073034638.py, line 16)

Step 2: Install Required Dependencies

In [ ]:
# Core dependencies installation
!pip install -q torch==2.1.0 torchvision torchaudio --index-url https://download.pytorch.
# Deep learning for graphs
!pip install -q dgl==1.1.3 dgllife==0.3.2
# PyTorch Geometric (alternative GNN library)
!pip install -q torch-geometric torch-scatter torch-sparse torch-cluster
# Cheminformatics
!pip install -q rdkit-pypi chembl_webresource_client biopython
# Protein language models
!pip install -q transformers==4.35.0 sentencepiece fair-esm
# DeepPurpose toolkit
!pip install -q DeepPurpose
# Data science and utilities
!pip install -q pandas==2.0.3 numpy==1.24.3 scikit-learn==1.3.0
!pip install -q matplotlib seaborn tqdm requests
# Explainability
!pip install -q shap captum
# Reproducibility and versioning
!pip install -q wandb mlflow pystow
print("✓ All dependencies installed successfully!")

Raw targets object from API: {'cross_references': [], 'organism': 'Homo sapiens', 'pref_name': 'Prothrombin', 'species_group_flag': False, 'target_chembl_id': 'CHEMBL204', 'target_components': [{'accession': 'P00734', 'component_description': 'Prothrombin', 'component_id': 92, 'component_type': 'PROTEIN', 'relationship': 'SINGLE PROTEIN', 'target_component_synonyms': [{'component_synonym': '3.4.21.5', 'syn_type': 'EC_NUMBER'}, {'component_synonym': 'Activation peptide fragment 1', 'syn_type': 'UNIPROT'}, {'component_synonym': 'Activation peptide fragment 2', 'syn_type': 'UNIPROT'}, {'component_synonym': 'Coagulation factor II', 'syn_type': 'UNIPROT'}, {'component_synonym': 'F2', 'syn_type': 'GENE_SYMBOL'}, {'component_synonym': 'Prothrombin', 'syn_type': 'UNIPROT'}, {'component_synonym': 'Thrombin heavy chain', 'syn_type': 'UNIPROT'}, {'component_synonym': 'Thrombin light chain', 'syn_type': 'UNIPROT'}], 'target_component_xrefs': [{'xref_id': '2H9T', 'xref_name': 'A', 'xref_src_db': 'P

Step 3: Set Random Seeds for Reproducibilit

In [ ]:
import random
import numpy as np
import torch
import dgl
def set_seed(seed=42):
"""
Set random seeds for reproducibility across all libraries.
Critical for ensuring deterministic results.
"""
random.seed(seed)
Step 2: Install Required Dependencies
Step 3: Set Random Seeds for Reproducibility
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
dgl.seed(seed)
dgl.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
print(f"✓ Random seed set to {seed} for reproducibility")
# Apply seed
RANDOM_SEED = 42
set_seed(RANDOM_SEED)


Scaffold Split Results:
Test Set Size: 1122 (20.23%)
Shared Scaffolds (must be 0): 0


Phase I: Integrated EDA and dataset splits for ChEMBL, DrugBank, and GDSC

Project initialization and dependencies

In [ ]:
# Minimal dependencies required for Phase I across all three datasets
import sys, subprocess, os

def pip_install(pkgs):
    for p in pkgs:
        print(f"Installing {p} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", p])

pip_install([
    "pandas==2.0.3",
    "numpy==1.24.3",
    "matplotlib==3.8.0",
    "seaborn==0.13.0",
    "scikit-learn==1.3.0",
    "tqdm==4.66.1",
    "rdkit-pypi",
    "chembl_webresource_client",
    "requests"
])

# Use your existing project root if already created in setup
PROJECT_DIR = "/content/DDLS_Drug_Repurposing"
os.makedirs(PROJECT_DIR, exist_ok=True)
os.chdir(PROJECT_DIR)

for d in ["data/raw", "data/processed", "results/figures", "results/metrics", "logs"]:
    os.makedirs(d, exist_ok=True)

print("Working directory:", os.getcwd())

In [ ]:
# Common imports, reproducibility, and plotting helpers
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    print(f"Random seed set to {seed}")

set_seed(42)

sns.set(style="whitegrid", context="notebook")
plt.rcParams["figure.dpi"] = 120

def save_fig(path):
    plt.tight_layout()
    plt.savefig(path, dpi=300, bbox_inches="tight")
    print(f"Saved figure: {path}")

ChEMBL dataset: acquisition, preprocessing, EDA, splits, and outputs

In [ ]:
# ChEMBL data acquisition and preprocessing for a single target (EGFR)
from chembl_webresource_client.new_client import new_client
from rdkit import Chem
from rdkit.Chem.Scaffolds import MurckoScaffold

TARGET_ID = "CHEMBL203"         # EGFR
PCHEMBL_THRESHOLD = 5.0         # active label threshold

def chembl_fetch_bioactivities(target_chembl_id: str) -> pd.DataFrame:
    print(f"Querying ChEMBL for binding assays: target={target_chembl_id} ...")
    df = pd.DataFrame.from_records(list(new_client.activity.filter(
        target_chembl_id=target_chembl_id,
        assay_type='B'
    )))
    print(f"Fetched records: {len(df)}")
    return df

def chembl_preprocess(df: pd.DataFrame, pchembl_threshold=PCHEMBL_THRESHOLD) -> pd.DataFrame:
    print("Preprocessing ChEMBL bioactivities ...")
    cols = [
        "molecule_chembl_id", "canonical_smiles", "standard_type", "standard_units",
        "standard_value", "pchembl_value", "target_chembl_id", "target_pref_name"
    ]
    cols = [c for c in cols if c in df.columns]
    df = df[cols].copy()

    df = df.dropna(subset=["canonical_smiles", "pchembl_value"])
    print(f"After NA drop: {len(df)} rows")

    valid_mask = []
    for s in tqdm(df["canonical_smiles"], desc="Validating SMILES"):
        valid_mask.append(Chem.MolFromSmiles(s) is not None)
    df = df.loc[valid_mask].reset_index(drop=True)
    print(f"Valid SMILES rows: {len(df)}")

    df["label"] = (df["pchembl_value"] >= pchembl_threshold).astype(int)
    df = df.drop_duplicates(subset=["canonical_smiles", "target_chembl_id"]).reset_index(drop=True)

    n_act = df["label"].sum()
    n_tot = len(df)
    print(f"Active: {n_act} ({n_act/n_tot:.2%}); Inactive: {n_tot - n_act}")
    return df

chembl_raw = chembl_fetch_bioactivities(TARGET_ID)
chembl_df = chembl_preprocess(chembl_raw)
chembl_out = "data/processed/chembl_egfr_processed.csv"
chembl_df.to_csv(chembl_out, index=False)
print("Saved:", chembl_out)
chembl_df.head()

In [ ]:
# ChEMBL EDA: overall distributions
print("ChEMBL shape:", chembl_df.shape)
print("Columns:", chembl_df.columns.tolist())
print("\nMissing values:\n", chembl_df.isna().sum().sort_values(ascending=False))

# Label distribution
plt.figure(figsize=(5, 4))
sns.countplot(x="label", data=chembl_df, palette="Set2")
plt.title("ChEMBL: Active vs Inactive (overall)")
plt.xlabel("Label (1=Active, 0=Inactive)")
plt.ylabel("Count")
save_fig("results/figures/chembl_labels_overall.png")
plt.show()

# pChEMBL histogram
if "pchembl_value" in chembl_df.columns:
    plt.figure(figsize=(6, 4))
    sns.histplot(chembl_df["pchembl_value"], bins=30, kde=True, color="#4C72B0")
    plt.axvline(PCHEMBL_THRESHOLD, color="red", linestyle="--", label=f"Threshold={PCHEMBL_THRESHOLD}")
    plt.title("ChEMBL: pChEMBL distribution (overall)")
    plt.xlabel("pChEMBL")
    plt.legend()
    save_fig("results/figures/chembl_pchembl_overall.png")
    plt.show()

# Standard value distribution if present
if "standard_value" in chembl_df.columns:
    plt.figure(figsize=(6, 4))
    sns.histplot(chembl_df["standard_value"].dropna(), bins=40, kde=True, color="#55A868")
    plt.title("ChEMBL: standard_value distribution")
    save_fig("results/figures/chembl_standard_value_overall.png")
    plt.show()

# Top standard types
if "standard_type" in chembl_df.columns:
    plt.figure(figsize=(8, 4))
    top_types = chembl_df["standard_type"].value_counts().head(10)
    sns.barplot(x=top_types.index, y=top_types.values, palette="Set3")
    plt.xticks(rotation=45, ha="right")
    plt.title("ChEMBL: top 10 standard_type")
    save_fig("results/figures/chembl_standard_type_top10.png")
    plt.show()

In [ ]:
# ChEMBL splits: stratified random split + scaffold-aware split
from sklearn.model_selection import train_test_split

X_smiles = chembl_df["canonical_smiles"].values
y_labels = chembl_df["label"].values
train_size, val_size, test_size = 0.8, 0.1, 0.1

# Stratified split
X_train, X_hold, y_train, y_hold = train_test_split(
    X_smiles, y_labels, test_size=(1 - train_size), random_state=42, stratify=y_labels
)
val_ratio = val_size / (val_size + test_size)
X_val, X_test, y_val, y_test = train_test_split(
    X_hold, y_hold, test_size=(1 - val_ratio), random_state=42, stratify=y_hold
)

df_c_train = chembl_df.loc[chembl_df["canonical_smiles"].isin(X_train)].copy()
df_c_val   = chembl_df.loc[chembl_df["canonical_smiles"].isin(X_val)].copy()
df_c_test  = chembl_df.loc[chembl_df["canonical_smiles"].isin(X_test)].copy()

df_c_train.to_csv("data/processed/chembl_train_stratified.csv", index=False)
df_c_val.to_csv("data/processed/chembl_val_stratified.csv", index=False)
df_c_test.to_csv("data/processed/chembl_test_stratified.csv", index=False)
print("ChEMBL stratified split saved.")

# Scaffold split
def scaffold(smiles: str):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None: return None
        return Chem.MolToSmiles(MurckoScaffold.GetScaffoldForMol(mol))
    except:
        return None

scaf_to_idx = {}
for i, s in enumerate(X_smiles):
    sc = scaffold(s)
    scaf_to_idx.setdefault(sc, []).append(i)

scaffolds = list(scaf_to_idx.keys())
sizes = [len(scaf_to_idx[s]) for s in scaffolds]
order = np.argsort(sizes)[::-1]

N = len(X_smiles)
train_lim = int(train_size * N)
val_lim = int(val_size * N)

train_idx, val_idx, test_idx = [], [], []
for k in order:
    grp = scaf_to_idx[scaffolds[k]]
    if len(train_idx) + len(grp) <= train_lim:
        train_idx.extend(grp)
    elif len(val_idx) + len(grp) <= val_lim:
        val_idx.extend(grp)
    else:
        test_idx.extend(grp)

rng = np.random.default_rng(42)
rng.shuffle(train_idx); rng.shuffle(val_idx); rng.shuffle(test_idx)

df_c_sc_train = chembl_df.iloc[train_idx].copy()
df_c_sc_val   = chembl_df.iloc[val_idx].copy()
df_c_sc_test  = chembl_df.iloc[test_idx].copy()

df_c_sc_train.to_csv("data/processed/chembl_train_scaffold.csv", index=False)
df_c_sc_val.to_csv("data/processed/chembl_val_scaffold.csv", index=False)
df_c_sc_test.to_csv("data/processed/chembl_test_scaffold.csv", index=False)
print("ChEMBL scaffold split saved.")

# Per-split visualizations
def plot_labels_per_split(df_train, df_val, df_test, name):
    plt.figure(figsize=(6, 4))
    data = pd.DataFrame({
        "split": ["train","train","val","val","test","test"],
        "label": [0,1,0,1,0,1],
        "count": [
            (df_train["label"]==0).sum(), (df_train["label"]==1).sum(),
            (df_val["label"]==0).sum(),   (df_val["label"]==1).sum(),
            (df_test["label"]==0).sum(),  (df_test["label"]==1).sum(),
        ]
    })
    sns.barplot(data=data, x="split", y="count", hue="label", palette="Set2")
    plt.title(f"{name}: label distribution per split")
    save_fig(f"results/figures/{name}_labels_per_split.png")
    plt.show()

def plot_pchembl_per_split(df_train, df_val, df_test, name):
    if "pchembl_value" not in df_train.columns: return
    fig, axes = plt.subplots(1, 3, figsize=(12, 3.5), sharey=True)
    sns.histplot(df_train["pchembl_value"], bins=30, kde=True, ax=axes[0], color="#4C72B0")
    axes[0].axvline(PCHEMBL_THRESHOLD, color="red", linestyle="--")
    axes[0].set_title("Train")
    sns.histplot(df_val["pchembl_value"], bins=30, kde=True, ax=axes[1], color="#55A868")
    axes[1].axvline(PCHEMBL_THRESHOLD, color="red", linestyle="--")
    axes[1].set_title("Val")
    sns.histplot(df_test["pchembl_value"], bins=30, kde=True, ax=axes[2], color="#C44E52")
    axes[2].axvline(PCHEMBL_THRESHOLD, color="red", linestyle="--")
    axes[2].set_title("Test")
    for ax in axes:
        ax.set_xlabel("pChEMBL"); ax.set_ylabel("Frequency")
    plt.suptitle(f"{name}: pChEMBL distributions per split", y=1.02)
    save_fig(f"results/figures/{name}_pchembl_per_split.png")
    plt.show()

# Stratified visuals
plot_labels_per_split(df_c_train, df_c_val, df_c_test, "chembl_stratified")
plot_pchembl_per_split(df_c_train, df_c_val, df_c_test, "chembl_stratified")

# Scaffold visuals
plot_labels_per_split(df_c_sc_train, df_c_sc_val, df_c_sc_test, "chembl_scaffold")
plot_pchembl_per_split(df_c_sc_train, df_c_sc_val, df_c_sc_test, "chembl_scaffold")

DrugBank dataset: ingestion, EDA, and outputs

In [ ]:
# DrugBank ingestion
# Assumption: You have a CSV/TSV exported file with basic fields.
# Update this path to your actual file:
DRUGBANK_PATH = "data/raw/drugbank.csv"  # e.g., contains columns like: name, drug_type, molecular_weight, targets, smiles

if not os.path.exists(DRUGBANK_PATH):
    print(f"WARNING: {DRUGBANK_PATH} not found. Please upload your DrugBank CSV to this path.")
else:
    drugbank_df = pd.read_csv(DRUGBANK_PATH)
    print("DrugBank shape:", drugbank_df.shape)
    print(drugbank_df.head())

    # Basic cleaning: drop obvious duplicates by SMILES/name if available
    key_cols = [c for c in ["smiles", "name"] if c in drugbank_df.columns]
    if key_cols:
        drugbank_df = drugbank_df.drop_duplicates(subset=key_cols).reset_index(drop=True)

    # Missing values report
    print("\nDrugBank missing values:\n", drugbank_df.isna().sum().sort_values(ascending=False).head(20))

    # Drug type distribution
    if "drug_type" in drugbank_df.columns:
        plt.figure(figsize=(8, 4))
        drugbank_df["drug_type"].value_counts().plot(kind="bar", color="#4C72B0")
        plt.title("DrugBank: drug_type distribution")
        plt.ylabel("Count")
        save_fig("results/figures/drugbank_drug_type.png")
        plt.show()

    # Molecular weight histogram
    if "molecular_weight" in drugbank_df.columns:
        plt.figure(figsize=(6, 4))
        sns.histplot(drugbank_df["molecular_weight"].dropna(), bins=50, kde=True, color="#55A868")
        plt.title("DrugBank: molecular weight distribution")
        save_fig("results/figures/drugbank_mw.png")
        plt.show()

    # Number of targets per drug
    if "targets" in drugbank_df.columns:
        tmp = drugbank_df["targets"].fillna("").astype(str)
        drugbank_df["n_targets"] = tmp.apply(lambda x: 0 if x.strip()=="" else len([t for t in x.split(";") if t]))
        plt.figure(figsize=(6, 4))
        sns.histplot(drugbank_df["n_targets"], bins=30, color="#C44E52")
        plt.title("DrugBank: number of targets per drug")
        save_fig("results/figures/drugbank_n_targets.png")
        plt.show()

    # Save processed
    drugbank_out = "data/processed/drugbank_processed.csv"
    drugbank_df.to_csv(drugbank_out, index=False)
    print("Saved:", drugbank_out)

In [ ]:
# -------------------------------
# DrugBank supervised proxy task
# -------------------------------

if 'drugbank_df' in globals() and 'mechanisms' in drugbank_df.columns:
    # 定义二分类标签：是否为 inhibitor
    drugbank_df['label'] = drugbank_df['mechanisms'].fillna("").str.lower().apply(
        lambda x: 1 if "inhibitor" in x else 0
    )

    print("DrugBank proxy labels created.")
    print(drugbank_df['label'].value_counts())

    # Stratified split
    from sklearn.model_selection import train_test_split

    X = drugbank_df.drop(columns=['label'])
    y = drugbank_df['label']

    X_train, X_hold, y_train, y_hold = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_hold, y_hold, test_size=0.5, random_state=42, stratify=y_hold
    )

    df_db_train = drugbank_df.iloc[X_train.index].copy()
    df_db_val   = drugbank_df.iloc[X_val.index].copy()
    df_db_test  = drugbank_df.iloc[X_test.index].copy()

    # 保存
    df_db_train.to_csv("data/processed/drugbank_train_stratified.csv", index=False)
    df_db_val.to_csv("data/processed/drugbank_val_stratified.csv", index=False)
    df_db_test.to_csv("data/processed/drugbank_test_stratified.csv", index=False)
    print("DrugBank stratified split saved.")

    # 可视化标签分布
    import seaborn as sns
    import matplotlib.pyplot as plt

    def plot_db_split(df_train, df_val, df_test, name="drugbank"):
        plt.figure(figsize=(6,4))
        data = pd.DataFrame({
            "split":["train","train","val","val","test","test"],
            "label":[0,1,0,1,0,1],
            "count":[
                (df_train['label']==0).sum(), (df_train['label']==1).sum(),
                (df_val['label']==0).sum(),   (df_val['label']==1).sum(),
                (df_test['label']==0).sum(),  (df_test['label']==1).sum(),
            ]
        })
        sns.barplot(data=data, x="split", y="count", hue="label", palette="Set2")
        plt.title(f"{name}: label distribution per split")
        save_fig(f"results/figures/{name}_labels_per_split.png")
        plt.show()

    plot_db_split(df_db_train, df_db_val, df_db_test)

GDSC dataset: download, preprocessing, EDA, splits, and outputs

In [ ]:
# GDSC download and EDA
import requests
from io import StringIO

GDSC_URL = "https://www.cancerrxgene.org/gdsc1000/GDSC1_fitted_dose_response.csv"
print("Downloading GDSC1 fitted dose-response (CSV) ...")
r = requests.get(GDSC_URL, timeout=60)
gdsc_df = pd.read_csv(StringIO(r.text))
print("GDSC shape:", gdsc_df.shape)
gdsc_df.head()

In [ ]:
# GDSC preprocessing
# Keep key columns; create sensitivity label from LN_IC50
keep = [c for c in ["DRUG_ID", "DRUG_NAME", "CELL_LINE_NAME", "COSMIC_ID", "LN_IC50", "AUC", "RMSE"] if c in gdsc_df.columns]
gdsc_df = gdsc_df[keep].copy()
gdsc_df = gdsc_df.dropna(subset=["LN_IC50"]).reset_index(drop=True)

SENS_THRESH = 2.0
gdsc_df["sensitive"] = (gdsc_df["LN_IC50"] < SENS_THRESH).astype(int)

print("GDSC rows after preprocessing:", len(gdsc_df))
print("Sensitive ratio:", gdsc_df["sensitive"].mean())

gdsc_out = "data/processed/gdsc_processed.csv"
gdsc_df.to_csv(gdsc_out, index=False)
print("Saved:", gdsc_out)

In [ ]:
# GDSC EDA
print("\nMissing values:\n", gdsc_df.isna().sum().sort_values(ascending=False).head(20))
print("Unique drugs:", gdsc_df["DRUG_NAME"].nunique() if "DRUG_NAME" in gdsc_df.columns else "N/A")
print("Unique cell lines:", gdsc_df["CELL_LINE_NAME"].nunique() if "CELL_LINE_NAME" in gdsc_df.columns else "N/A")

# LN_IC50 distribution
plt.figure(figsize=(6,4))
sns.histplot(gdsc_df["LN_IC50"], bins=50, kde=True, color="#4C72B0")
plt.title("GDSC: LN(IC50) distribution")
plt.xlabel("LN(IC50)")
save_fig("results/figures/gdsc_ln_ic50.png")
plt.show()

# AUC distribution
if "AUC" in gdsc_df.columns:
    plt.figure(figsize=(6,4))
    sns.histplot(gdsc_df["AUC"].dropna(), bins=50, kde=True, color="#55A868")
    plt.title("GDSC: AUC distribution")
    plt.xlabel("AUC")
    save_fig("results/figures/gdsc_auc.png")
    plt.show()

# Sensitive vs not
plt.figure(figsize=(5,4))
sns.countplot(x="sensitive", data=gdsc_df, palette="Set2")
plt.title("GDSC: label distribution (sensitive)")
save_fig("results/figures/gdsc_sensitive_overall.png")
plt.show()

# Top drugs by tested cell lines
if "DRUG_NAME" in gdsc_df.columns:
    top_drugs = gdsc_df["DRUG_NAME"].value_counts().head(20)
    plt.figure(figsize=(9,4))
    sns.barplot(x=top_drugs.index, y=top_drugs.values, palette="Set3")
    plt.xticks(rotation=90)
    plt.title("GDSC: top 20 drugs by number of tested cell lines")
    save_fig("results/figures/gdsc_top_drugs.png")
    plt.show()

In [ ]:
# GDSC splits (stratified)
from sklearn.model_selection import train_test_split

X = gdsc_df[["DRUG_NAME", "CELL_LINE_NAME"]] if "DRUG_NAME" in gdsc_df.columns and "CELL_LINE_NAME" in gdsc_df.columns else gdsc_df[["LN_IC50"]]
y = gdsc_df["sensitive"]

X_train, X_hold, y_train, y_hold = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_hold, y_hold, test_size=0.5, random_state=42, stratify=y_hold)

df_g_train = gdsc_df.iloc[X_train.index].copy()
df_g_val   = gdsc_df.iloc[X_val.index].copy()
df_g_test  = gdsc_df.iloc[X_test.index].copy()

df_g_train.to_csv("data/processed/gdsc_train_stratified.csv", index=False)
df_g_val.to_csv("data/processed/gdsc_val_stratified.csv", index=False)
df_g_test.to_csv("data/processed/gdsc_test_stratified.csv", index=False)
print("GDSC stratified split saved.")

# Per-split label bars
def plot_binary_split_bars(df_train, df_val, df_test, label_col, name):
    plt.figure(figsize=(6,4))
    data = pd.DataFrame({
        "split":["train","train","val","val","test","test"],
        "label":[0,1,0,1,0,1],
        "count":[
            (df_train[label_col]==0).sum(), (df_train[label_col]==1).sum(),
            (df_val[label_col]==0).sum(),   (df_val[label_col]==1).sum(),
            (df_test[label_col]==0).sum(),  (df_test[label_col]==1).sum(),
        ]
    })
    sns.barplot(data=data, x="split", y="count", hue="label", palette="Set2")
    plt.title(f"{name}: label distribution per split")
    save_fig(f"results/figures/{name}_labels_per_split.png")
    plt.show()

plot_binary_split_bars(df_g_train, df_g_val, df_g_test, "sensitive", "gdsc_stratified")

Consolidated summary and Phase I checklist

In [ ]:
# Summaries for ChEMBL splits
def summarize(df, label_col, split_name, dataset_name):
    n = len(df); n_pos = int(df[label_col].sum())
    return {
        "dataset": dataset_name,
        "split": split_name,
        "n_samples": n,
        "n_positive": n_pos,
        "positive_ratio": round(n_pos / n, 4) if n > 0 else 0.0
    }

# -------------------------------
# Consolidated summary
# -------------------------------

summary_rows = []

# ChEMBL (stratified + scaffold)
summary_rows += [
    summarize(df_c_train, "label", "train", "chembl_stratified"),
    summarize(df_c_val,   "label", "val",   "chembl_stratified"),
    summarize(df_c_test,  "label", "test",  "chembl_stratified"),
    summarize(df_c_sc_train, "label", "train", "chembl_scaffold"),
    summarize(df_c_sc_val,   "label", "val",   "chembl_scaffold"),
    summarize(df_c_sc_test,  "label", "test",  "chembl_scaffold"),
]

# GDSC (stratified)
summary_rows += [
    summarize(df_g_train, "sensitive", "train", "gdsc_stratified"),
    summarize(df_g_val,   "sensitive", "val",   "gdsc_stratified"),
    summarize(df_g_test,  "sensitive", "test",  "gdsc_stratified"),
]

# DrugBank (proxy supervised, stratified)
if 'df_db_train' in globals():
    summary_rows += [
        summarize(df_db_train, "label", "train", "drugbank_stratified"),
        summarize(df_db_val,   "label", "val",   "drugbank_stratified"),
        summarize(df_db_test,  "label", "test",  "drugbank_stratified"),
    ]

summary_df = pd.DataFrame(summary_rows)
summary_out = "results/metrics/phase1_summary.csv"
summary_df.to_csv(summary_out, index=False)
print("Saved updated summary:", summary_out)
summary_df

In [ ]:
# Phase I deliverable checklist
checks = [
    ("ChEMBL overall labels", "results/figures/chembl_labels_overall.png"),
    ("ChEMBL overall pChEMBL", "results/figures/chembl_pchembl_overall.png"),
    ("ChEMBL stratified label per split", "results/figures/chembl_stratified_labels_per_split.png"),
    ("ChEMBL stratified pChEMBL per split", "results/figures/chembl_stratified_pchembl_per_split.png"),
    ("ChEMBL scaffold label per split", "results/figures/chembl_scaffold_labels_per_split.png"),
    ("ChEMBL scaffold pChEMBL per split", "results/figures/chembl_scaffold_pchembl_per_split.png"),
    ("DrugBank type distribution", "results/figures/drugbank_drug_type.png"),
    ("DrugBank molecular weight", "results/figures/drugbank_mw.png"),
    ("DrugBank #targets", "results/figures/drugbank_n_targets.png"),
    ("GDSC LN(IC50) histogram", "results/figures/gdsc_ln_ic50.png"),
    ("GDSC AUC histogram", "results/figures/gdsc_auc.png"),
    ("GDSC stratified split labels", "results/figures/gdsc_stratified_labels_per_split.png"),
    ("Phase I summary CSV", "results/metrics/phase1_summary.csv"),
]

for label, path in checks:
    print(f"- {label}: {'OK' if os.path.exists(path) else 'MISSING'}")

Phase II: Model training, evaluation, and improvement

Baseline training (fast, reproducible): Establish a quick baseline using classical ML on Morgan fingerprints from ChEMBL (e.g., Random Forest), then extend to GDSC proxy tasks and DrugBank proxy label.

In [ ]:
# 1) Load processed ChEMBL splits (stratified)
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.model_selection import GridSearchCV
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np

train = pd.read_csv("data/processed/chembl_train_stratified.csv")
val   = pd.read_csv("data/processed/chembl_val_stratified.csv")
test  = pd.read_csv("data/processed/chembl_test_stratified.csv")

def morgan_fp(smiles, radius=2, n_bits=2048):
    v = np.zeros((len(smiles), n_bits), dtype=np.int8)
    for i, s in enumerate(smiles):
        mol = Chem.MolFromSmiles(s)
        if mol:
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
            arr = np.zeros((n_bits,), dtype=np.int8)
            AllChem.DataStructs.ConvertToNumpyArray(fp, arr)
            v[i] = arr
    return v

X_train = morgan_fp(train["canonical_smiles"].values)
y_train = train["label"].values
X_val   = morgan_fp(val["canonical_smiles"].values)
y_val   = val["label"].values
X_test  = morgan_fp(test["canonical_smiles"].values)
y_test  = test["label"].values

# 2) Train baseline RF + tune key hyperparameters
rf = RandomForestClassifier(n_estimators=500, max_depth=20, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
val_pred = rf.predict_proba(X_val)[:, 1]
print("Val ROC-AUC:", roc_auc_score(y_val, val_pred))
print("Val PR-AUC:", average_precision_score(y_val, val_pred))

# Optional: small grid search
param_grid = {"n_estimators":[300,500,800], "max_depth":[10,20,None]}
search = GridSearchCV(RandomForestClassifier(random_state=42, n_jobs=-1),
                      param_grid, scoring="roc_auc", cv=3, n_jobs=-1)
search.fit(X_train, y_train)
best = search.best_estimator_
print("Best params:", search.best_params_)
val_pred = best.predict_proba(X_val)[:, 1]
print("Val ROC-AUC (best):", roc_auc_score(y_val, val_pred))

In [ ]:
from sklearn.metrics import precision_recall_curve, roc_curve
import json

test_pred = best.predict_proba(X_test)[:, 1]
metrics = {
    "roc_auc": roc_auc_score(y_test, test_pred),
    "pr_auc": average_precision_score(y_test, test_pred)
}
with open("results/metrics/chembl_baseline_metrics.json", "w") as f:
    json.dump(metrics, f, indent=2)
print("Saved:", "results/metrics/chembl_baseline_metrics.json")

Extension to GDSC (binary “sensitive” label) and DrugBank (proxy label)

GDSC: Use features like LN_IC50, AUC, or simple bag-of-entities (drug name, cell line) with target encoding for a fast baseline.

In [ ]:
# GDSC baseline (e.g., simple numeric features)
gdsc_train = pd.read_csv("data/processed/gdsc_train_stratified.csv")
gdsc_val   = pd.read_csv("data/processed/gdsc_val_stratified.csv")
gdsc_test  = pd.read_csv("data/processed/gdsc_test_stratified.csv")

feat_cols = [c for c in ["LN_IC50","AUC","RMSE"] if c in gdsc_train.columns]
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=500, class_weight="balanced"))
])

pipe.fit(gdsc_train[feat_cols], gdsc_train["sensitive"])
val_pred = pipe.predict_proba(gdsc_val[feat_cols])[:,1]
print("GDSC Val ROC-AUC:", roc_auc_score(gdsc_val["sensitive"], val_pred))
print("GDSC Val PR-AUC:", average_precision_score(gdsc_val["sensitive"], val_pred))

DrugBank proxy: Use simple textual features (mechanisms, drug_type) via TF-IDF for a baseline text classifier.

In [ ]:
# DrugBank text proxy (mechanisms -> inhibitor label)
db_train = pd.read_csv("data/processed/drugbank_train_stratified.csv")
db_val   = pd.read_csv("data/processed/drugbank_val_stratified.csv")
db_test  = pd.read_csv("data/processed/drugbank_test_stratified.csv")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

tfidf = TfidfVectorizer(min_df=3, ngram_range=(1,2))
Xtr = tfidf.fit_transform(db_train["mechanisms"].fillna(""))
Xva = tfidf.transform(db_val["mechanisms"].fillna(""))
Xte = tfidf.transform(db_test["mechanisms"].fillna(""))

lr = LogisticRegression(max_iter=1000, class_weight="balanced")
lr.fit(Xtr, db_train["label"])
val_pred = lr.predict_proba(Xva)[:,1]
print("DrugBank Val ROC-AUC:", roc_auc_score(db_val["label"], val_pred))
print("DrugBank Val PR-AUC:", average_precision_score(db_val["label"], val_pred))

Improvement loop: Ideas are class weighting, threshold tuning via PR curve, modest feature selection, scaffold split comparison for ChEMBL, early error analysis (false positives/negatives), ablations.

In [ ]:
precision, recall, thresh = precision_recall_curve(y_val, val_pred)
# Choose threshold maximizing F1
f1 = 2 * precision * recall / (precision + recall + 1e-9)
best_t = thresh[np.argmax(f1[:-1])]
print("Best threshold by F1:", best_t)

Phase III: Minimal accessibility toolset (Option B, MCP-like)

Create small, composable functions to load data, run inference, show results, and evaluate. Then wire them into a simple CLI or notebook widgets.

In [ ]:
# tools/data_io.py-like functions (define in notebook cells or src/utils)
import pandas as pd

def load_split(dataset, split):
    path = f"data/processed/{dataset}_{split}_stratified.csv"
    return pd.read_csv(path)

def load_model_artifact(dataset):
    # For baseline demo, re-fit quickly or load from disk if saved
    import joblib, os
    p = f"models/baseline/{dataset}_model.pkl"
    return joblib.load(p) if os.path.exists(p) else None

In [ ]:
# tools/inference.py-like functions
def infer_chembl(model, smiles_list):
    X = morgan_fp(smiles_list)
    return model.predict_proba(X)[:,1]

def infer_drugbank(model, mechanisms_list, tfidf):
    X = tfidf.transform(mechanisms_list)
    return model.predict_proba(X)[:,1]

def infer_gdsc(model, df, feat_cols):
    return model.predict_proba(df[feat_cols])[:,1]

In [ ]:
# tools/evaluate.py-like functions
from sklearn.metrics import roc_auc_score, average_precision_score

def evaluate_binary(y_true, y_pred, name, save_path=None):
    m = {"roc_auc": roc_auc_score(y_true, y_pred),
         "pr_auc": average_precision_score(y_true, y_pred)}
    if save_path:
        import json
        with open(save_path, "w") as f: json.dump(m, f, indent=2)
    print(f"[{name}] ROC-AUC={m['roc_auc']:.4f} PR-AUC={m['pr_auc']:.4f}")
    return m

Lightweight CLI-like cell (data → model → inference → eval)

Keep everything callable with 3–4 function calls; mirror computer lab styles with simple parameterization.

In [ ]:
# Example: run end-to-end for ChEMBL
train, val, test = load_split("chembl", "train"), load_split("chembl", "val"), load_split("chembl", "test")
Xtr, ytr = morgan_fp(train["canonical_smiles"].values), train["label"].values
Xva, yva = morgan_fp(val["canonical_smiles"].values), val["label"].values

rf = RandomForestClassifier(n_estimators=500, max_depth=20, random_state=42, n_jobs=-1).fit(Xtr, ytr)
va_pred = rf.predict_proba(Xva)[:,1]
evaluate_binary(yva, va_pred, "chembl_val", "results/metrics/chembl_val_baseline.json")